<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#读入数据集" data-toc-modified-id="读入数据集-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>读入数据集</a></span></li><li><span><a href="#构建用户和物品之间的关系矩阵" data-toc-modified-id="构建用户和物品之间的关系矩阵-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>构建用户和物品之间的关系矩阵</a></span><ul class="toc-item"><li><span><a href="#将矩阵中的空值填充成0" data-toc-modified-id="将矩阵中的空值填充成0-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>将矩阵中的空值填充成0</a></span></li></ul></li><li><span><a href="#计算用户相似度(余弦相似度)" data-toc-modified-id="计算用户相似度(余弦相似度)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>计算用户相似度(余弦相似度)</a></span><ul class="toc-item"><li><span><a href="#计算皮尔逊相关性" data-toc-modified-id="计算皮尔逊相关性-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>计算皮尔逊相关性</a></span></li></ul></li><li><span><a href="#调用计算推荐指数函数cal_recommend_index" data-toc-modified-id="调用计算推荐指数函数cal_recommend_index-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>调用计算推荐指数函数cal_recommend_index</a></span></li><li><span><a href="#构建预测函数" data-toc-modified-id="构建预测函数-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>构建预测函数</a></span><ul class="toc-item"><li><span><a href="#直接调用写好的函数" data-toc-modified-id="直接调用写好的函数-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>直接调用写好的函数</a></span></li></ul></li><li><span><a href="#构建最终的推荐功能" data-toc-modified-id="构建最终的推荐功能-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>构建最终的推荐功能</a></span><ul class="toc-item"><li><span><a href="#对上面的数据进行分组操作" data-toc-modified-id="对上面的数据进行分组操作-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>对上面的数据进行分组操作</a></span></li><li><span><a href="#对函数进行测试" data-toc-modified-id="对函数进行测试-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>对函数进行测试</a></span></li><li><span><a href="#用apply调用上面的函数进行分组操作" data-toc-modified-id="用apply调用上面的函数进行分组操作-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>用apply调用上面的函数进行分组操作</a></span></li></ul></li></ul></div>

# 手写实现协同过滤算法

因为算法比较困难, 所以我们使用课件中的简单数据集先去进行模型试验.

In [1]:
import numpy as np
import pandas as pd

## 读入数据集

In [2]:
# 把课件中的example.txt 读入进来
df = pd.read_csv('example.txt',header = None )
# header = None 不将数据集中的第一列作为列名读取
# 给读进来的表格设置一个列名, 重新传递列名
df.columns = ['用户ID','产品ID','评分']
df

,用户ID,产品ID,评分
0,1,1,4
1,1,2,3
2,1,5,5
3,2,1,5
4,2,3,4
5,2,5,4
6,3,1,4
7,3,3,5
8,3,4,3
9,3,5,4


## 构建用户和物品之间的关系矩阵

index=None, columns=None, values=None

index ,用哪里列作为行

columns, 用哪个作为透视表的列

values, 用那一列作为值

In [3]:
# 使用的是透视表函数
df_pivot = df.pivot(index='用户ID',columns='产品ID',values= '评分')

In [4]:
df_pivot

产品ID,1,2,3,4,5,6
用户ID,,,,,,
1,4.0,3.0,NaN,NaN,5.0,NaN
2,5.0,NaN,4.0,NaN,4.0,NaN
3,4.0,NaN,5.0,3.0,4.0,NaN
4,NaN,3.0,NaN,NaN,NaN,5.0
5,NaN,4.0,NaN,NaN,NaN,4.0
6,NaN,NaN,2.0,4.0,NaN,5.0


### 将矩阵中的空值填充成0

In [5]:
# inplace = True 直接在原数据集中进行替换
df_pivot.fillna(0 , inplace=True)

In [30]:
df_pivot

产品ID,1,2,3,4,5,6
用户ID,,,,,,
1,4.0,3.0,0.0,0.0,5.0,0.0
2,5.0,0.0,4.0,0.0,4.0,0.0
3,4.0,0.0,5.0,3.0,4.0,0.0
4,0.0,3.0,0.0,0.0,0.0,5.0
5,0.0,4.0,0.0,0.0,0.0,4.0
6,0.0,0.0,2.0,4.0,0.0,5.0


In [7]:
# 将上面的DataFrame 转换成数组
freq_matrix = df_pivot.values
freq_matrix

array([[4., 3., 0., 0., 5., 0.],
       [5., 0., 4., 0., 4., 0.],
       [4., 0., 5., 3., 4., 0.],
       [0., 3., 0., 0., 0., 5.],
       [0., 4., 0., 0., 0., 4.],
       [0., 0., 2., 4., 0., 5.]])

## 计算用户相似度(余弦相似度)

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
# 将用户相似度计算出来并进行保存
user_similar_matrix = cosine_similarity(freq_matrix)
user_similar_matrix

array([[1.        , 0.74926865, 0.62667956, 0.21828206, 0.3       ,
        0.        ],
       [0.74926865, 1.        , 0.91301651, 0.        , 0.        ,
        0.1579597 ],
       [0.62667956, 0.91301651, 1.        , 0.        , 0.        ,
        0.40368671],
       [0.21828206, 0.        , 0.        , 1.        , 0.9701425 ,
        0.63913749],
       [0.3       , 0.        , 0.        , 0.9701425 , 1.        ,
        0.52704628],
       [0.        , 0.1579597 , 0.40368671, 0.63913749, 0.52704628,
        1.        ]])

### 计算皮尔逊相关性

因为corr默认计算的是列之间的相关性,而我们要计算的是行之间的相关性, 那么我们先对矩阵进行转置, 将行列进行调换, 然后再计算

In [10]:
# 皮尔孙相关系数的计算
df_pivot.T.corr()

用户ID,1,2,3,4,5,6
用户ID,,,,,,
1,1.000000,0.511321,0.162400,-0.284199,-0.169842,-0.865802
2,0.511321,1.000000,0.822475,-0.668261,-0.698884,-0.591708
3,0.162400,0.822475,1.000000,-0.914286,-0.956183,-0.304646
4,-0.284199,-0.668261,-0.914286,1.000000,0.956183,0.429274
5,-0.169842,-0.698884,-0.956183,0.956183,1.000000,0.231714
6,-0.865802,-0.591708,-0.304646,0.429274,0.231714,1.000000


## 调用计算推荐指数函数cal_recommend_index

函数功能: 传入用户ID , 和产品ID ,以及最相似的k个用户,返回该产品对该用户的推荐指数

In [11]:
from recommend import cal_recommend_index_by_users

In [12]:
# 这个代表 对于 用户0 ,计算产品2 对他的推荐指数
cal_recommend_index_by_users(k = 2,user_id= 0,item_id = 2, 
            freq_matrix=freq_matrix , 
            user_similar_matrix=user_similar_matrix)

4.455452869929619

## 构建预测函数

1. 先构建一个空的矩阵, 用户填充用户和产品之间的推荐指数是
2. 构建的这个矩阵必须是和freq_matrix这个矩阵形状相同
3. 写一个两层循环, 分别对矩阵中的值进行运算, 采用刚才写好的cal_recommend_index函数进行计算其中的值.
4. 前提条件: 如果原来freq_matrix中的值已经有了,则不需要计算,如果值为0,则计算这个值,并填充到构建的空矩阵当中去

np.zeros_like: 函数功能, 构建一个0矩阵,形状和传入的矩阵相同

第一个参数a: 传入的这个矩阵

In [13]:
predict_matrix = np.zeros_like(freq_matrix)
predict_matrix

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [14]:
# 写一个两层循环, 分别对predict_matrix 矩阵中的每一个位置的值进行遍历
# predict_matrix.shape[0] 代表用户个数 , range()生成一个线性序列和这个长度相同
for user_id in range(predict_matrix.shape[0]):
    # 对每行中的每一个值进行遍历
    for item_id in range(predict_matrix.shape[1]):
        # 首先判断原来这个用户有没有对这个产品评分过,如果有不需要计算,如果没有 值为0,则进行计算
        if freq_matrix[user_id,item_id ] == 0:
            # 如果为0 计算推荐指数,
            recommend_index= cal_recommend_index_by_users(2, user_id, item_id 
                                ,freq_matrix,user_similar_matrix)
            # 算出来之后, 把这个数值保存到矩阵中去
            predict_matrix[user_id , item_id] = recommend_index
            

In [15]:
predict_matrix

array([[0.        , 0.        , 4.45545287, 3.        , 0.        ,
        0.        ],
       [0.        , 3.        , 0.        , 3.        , 0.        ,
        0.        ],
       [0.        , 3.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.        , 4.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.        , 4.        , 0.        ,
        0.        ],
       [0.        , 3.45194102, 0.        , 0.        , 0.        ,
        0.        ]])

### 直接调用写好的函数

In [16]:
from recommend import cal_recommend_index_matrix

In [17]:
# 调用上面的函数计算推荐系数矩阵
predict_matrix = cal_recommend_index_matrix(freq_matrix, user_similar_matrix)
predict_matrix

array([[0.        , 0.        , 4.45545287, 3.        , 0.        ,
        0.        ],
       [0.        , 3.        , 0.        , 3.        , 0.        ,
        0.        ],
       [0.        , 3.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.        , 4.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.        , 4.        , 0.        ,
        0.        ],
       [0.        , 3.45194102, 0.        , 0.        , 0.        ,
        0.        ]])

## 构建最终的推荐功能

1. 首先把上面的矩阵转换成DataFrame,方便操作
2. 现在的推荐矩阵不太方便操作,我们把数据转换回最初的那种格式
```
re_df_2 = re_df.stack().reset_index()
re_df_2
```
3. 现在我们需要根据每一个用户ID,提取推荐指数最大的两个产品
4. 所以我们需要对数据进行分组, 根据用户ID进行分组.
5. 如何提取出最大的两个数据呢? 默认没有这样功能, 所以我们自己构建一个函数,然后使用分组中的apply功能 , 应用这个聚合函数, 得到最终推荐结果.
6. 清理数据中的无效信息

In [36]:
df_pivot

产品ID,1,2,3,4,5,6
用户ID,,,,,,
1,4.0,3.0,0.0,0.0,5.0,0.0
2,5.0,0.0,4.0,0.0,4.0,0.0
3,4.0,0.0,5.0,3.0,4.0,0.0
4,0.0,3.0,0.0,0.0,0.0,5.0
5,0.0,4.0,0.0,0.0,0.0,4.0
6,0.0,0.0,2.0,4.0,0.0,5.0


In [37]:
# 把这个推荐矩阵转换成DataFrame, 索引值使用之前的那个df_pivot的索引值
re_df = pd.DataFrame(predict_matrix ,index=df_pivot.index , 
             columns= df_pivot.columns)
re_df

产品ID,1,2,3,4,5,6
用户ID,,,,,,
1,0.0,0.000000,4.455453,3.0,0.0,0.0
2,0.0,3.000000,0.000000,3.0,0.0,0.0
3,0.0,3.000000,0.000000,0.0,0.0,0.0
4,0.0,0.000000,2.000000,4.0,0.0,0.0
5,0.0,0.000000,2.000000,4.0,0.0,0.0
6,0.0,3.451941,0.000000,0.0,0.0,0.0


In [38]:
# 把二维数据DataFrame 转化成 2重索引的 一维Series
re_df.stack()

用户ID  产品ID
1     1       0.000000
      2       0.000000
      3       4.455453
      4       3.000000
      5       0.000000
      6       0.000000
2     1       0.000000
      2       3.000000
      3       0.000000
      4       3.000000
      5       0.000000
      6       0.000000
3     1       0.000000
      2       3.000000
      3       0.000000
      4       0.000000
      5       0.000000
      6       0.000000
4     1       0.000000
      2       0.000000
      3       2.000000
      4       4.000000
      5       0.000000
      6       0.000000
5     1       0.000000
      2       0.000000
      3       2.000000
      4       4.000000
      5       0.000000
      6       0.000000
6     1       0.000000
      2       3.451941
      3       0.000000
      4       0.000000
      5       0.000000
      6       0.000000
dtype: float64

In [39]:
# 对多重索引的Series进行 reset_index , 就会把索引和列都变成DataFrame里面的数据
re_df_2 = re_df.stack().reset_index()
re_df_2

,用户ID,产品ID,0
0,1,1,0.000000
1,1,2,0.000000
2,1,3,4.455453
3,1,4,3.000000
4,1,5,0.000000
5,1,6,0.000000
6,2,1,0.000000
7,2,2,3.000000
8,2,3,0.000000
9,2,4,3.000000


In [40]:
# 把上面的列名0 改成 推荐指数
re_df_2.rename(columns={0:"推荐指数"} ,inplace=True)

In [41]:
re_df_2

,用户ID,产品ID,推荐指数
0,1,1,0.000000
1,1,2,0.000000
2,1,3,4.455453
3,1,4,3.000000
4,1,5,0.000000
5,1,6,0.000000
6,2,1,0.000000
7,2,2,3.000000
8,2,3,0.000000
9,2,4,3.000000


### 对上面的数据进行分组操作

以用户ID进行分组, 提取出每组中最大的两个推荐指数的数据

In [42]:
# 保存分组结果, 但是先不进行合并
re_df_grouped = re_df_2.groupby(by='用户ID')
re_df_grouped

In [43]:
# 定义函数功能, 参数n代表要提取几个东西
# 第一个参数分组结果
def get_topn(group, n):
    # group 就代表了每一组数据
    # 计算出每一组数据中的最大的两个数据的id 和指数
    # 首先对每组数据进行排序, 排序后切片出前两个最大的值
    r = group.sort_values(by='推荐指数', ascending=False)[:n]
    return r

### 对函数进行测试

In [44]:
# 对上面的函数功能进行一个测试, 传入所有用户ID为1的数据
re_df_2[re_df_2.用户ID == 1]

,用户ID,产品ID,推荐指数
0,1,1,0.000000
1,1,2,0.000000
2,1,3,4.455453
3,1,4,3.000000
4,1,5,0.000000
5,1,6,0.000000


In [45]:
# 就会获得前两个最大值
get_topn(re_df_2[re_df_2.用户ID == 1], n = 2)

,用户ID,产品ID,推荐指数
2,1,3,4.455453
3,1,4,3.000000


### 用apply调用上面的函数进行分组操作

In [50]:
# 我们使用 分组中的apply功能, 这个功能可以自定义函数
topn = re_df_grouped.apply(get_topn, n = 2)
topn

用户ID  产品ID      推荐指数
用户ID                         
1    2      1     3  4.455453
     3      1     4  3.000000
2    7      2     2  3.000000
     9      2     4  3.000000
3    13     3     2  3.000000
     12     3     1  0.000000
4    21     4     4  4.000000
     20     4     3  2.000000
5    27     5     4  4.000000
     26     5     3  2.000000
6    31     6     2  3.451941
     30     6     1  0.000000

In [51]:
# 上面的结果多了一个无用的索引, 把它删除掉
# 把删除后的新的索引传递给这个表
topn.index = topn.index.droplevel(1)

In [52]:
# 得到最终的推荐结果
topn

,用户ID,产品ID,推荐指数
用户ID,,,
1,1,3,4.455453
1,1,4,3.000000
2,2,2,3.000000
2,2,4,3.000000
3,3,2,3.000000
3,3,1,0.000000
4,4,4,4.000000
4,4,3,2.000000
5,5,4,4.000000


In [53]:
topn.drop( columns='用户ID',inplace = True)

In [54]:
topn

,产品ID,推荐指数
用户ID,,
1,3,4.455453
1,4,3.000000
2,2,3.000000
2,4,3.000000
3,2,3.000000
3,1,0.000000
4,4,4.000000
4,3,2.000000
5,4,4.000000


In [55]:
# 和原来的相似度矩阵做一个对比
predict_matrix

array([[0.        , 0.        , 4.45545287, 3.        , 0.        ,
        0.        ],
       [0.        , 3.        , 0.        , 3.        , 0.        ,
        0.        ],
       [0.        , 3.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.        , 4.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.        , 4.        , 0.        ,
        0.        ],
       [0.        , 3.45194102, 0.        , 0.        , 0.        ,
        0.        ]])

In [57]:
def get_recom( predict_matrix , k = 2):
    # 把这个推荐矩阵转换成DataFrame, 索引值使用之前的那个df_pivot的索引值
    re_df = pd.DataFrame(predict_matrix ,index=df_pivot.index , 
                 columns= df_pivot.columns)
    # 对多重索引的Series进行 reset_index , 就会把索引和列都变成DataFrame里面的数据
    re_df_2 = re_df.stack().reset_index()
    # 把上面的列名0 改成 推荐指数
    re_df_2.rename(columns={0:"推荐指数"} ,inplace=True)
    # 保存分组结果, 但是先不进行合并
    re_df_grouped = re_df_2.groupby(by='用户ID')
    # 定义函数功能, 参数n代表要提取几个东西
    # 第一个参数分组结果
    def get_topn(group, n):
        # group 就代表了每一组数据
        # 计算出每一组数据中的最大的两个数据的id 和指数
        # 首先对每组数据进行排序, 排序后切片出前两个最大的值
        r = group.sort_values(by='推荐指数', ascending=False)[:n]
        return r
    # 我们使用 分组中的apply功能, 这个功能可以自定义函数
    topn = re_df_grouped.apply(get_topn, n = 2)
    # 上面的结果多了一个无用的索引, 把它删除掉
    # 把删除后的新的索引传递给这个表
    topn.index = topn.index.droplevel(1)
    topn.drop( columns='用户ID',inplace = True)
    return topn

In [58]:
get_recom(predict_matrix , k = 2)

,产品ID,推荐指数
用户ID,,
1,3,4.455453
1,4,3.000000
2,2,3.000000
2,4,3.000000
3,2,3.000000
3,1,0.000000
4,4,4.000000
4,3,2.000000
5,4,4.000000
